# Netflix Show Cancellations: Reddit API Analysis TEST


## Step 1: Install Required Libraries + Set Up the Reddit API Connection
Before using the Reddit API, we need to install the `praw` library. It simplifies authentication and API interactions.

In [4]:
# Install the required library
!pip install praw

In [6]:
import praw
print("praw library imported successfully!")


praw library imported successfully!


In [8]:
!pip install python-dotenv
#This is needed to create an .env for your API credentials. We dont want random people having access to our keys!

In [9]:
import praw
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Retrieve credentials from environment variables
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
USER_AGENT = os.getenv('USER_AGENT')

# Authenticate with Reddit API
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

# Test connection
print("Reddit API is read-only:", reddit.read_only)

Reddit API is read-only: True


## Step 2: Test Access to the Subreddits
verify that we can access the two subreddits (`r/NetflixBestOf` and `r/television`) using the Reddit API.

In [10]:
# Test access to the subreddits
subreddits_to_test = ['NetflixBestOf', 'television']

for subreddit_name in subreddits_to_test:
    try:
        subreddit = reddit.subreddit(subreddit_name)  # Access subreddit
        print(f"Successfully accessed subreddit: {subreddit.display_name}")
    except Exception as e:
        print(f"Error accessing subreddit {subreddit_name}: {e}")

Successfully accessed subreddit: NetflixBestOf
Successfully accessed subreddit: television


### Step 2.1: Search for Posts with "Mindhunter" in the Title (Scenario 2)
We will search the two subreddits for posts that include the title of the show ("Mindhunter") in their post titles.


In [11]:
import pandas as pd
# Search for posts with "Mindhunter" in the title
show_title = "Mindhunter"  # The show we're testing
subreddits_to_test = ['NetflixBestOf', 'television']
posts_with_title = []  # List to store results

for subreddit_name in subreddits_to_test:
    print(f"Searching in subreddit: {subreddit_name}")
    try:
        subreddit = reddit.subreddit(subreddit_name)
        for post in subreddit.search(show_title, limit=10):  # Limit to 10 posts
            posts_with_title.append({
                'Subreddit': subreddit_name,
                'Title': post.title,
                'Num_Comments': post.num_comments,
                'Upvotes': post.score,
                'Post_ID': post.id,
                'Created_UTC': post.created_utc
            })
    except Exception as e:
        print(f"Error searching subreddit {subreddit_name}: {e}")

# Convert results to DataFrame for easier analysis
if posts_with_title:
    posts_df = pd.DataFrame(posts_with_title)
    print(f"Found {len(posts_df)} posts with '{show_title}' in the title:")
    display(posts_df)
else:
    print(f"No posts with '{show_title}' in the title found.")


Searching in subreddit: NetflixBestOf
Searching in subreddit: television
Found 20 posts with 'Mindhunter' in the title:


,Subreddit,Title,Num_Comments,Upvotes,Post_ID,Created_UTC
0,NetflixBestOf,[US] MINDHUNTER (2017): honest opinion is it s...,225,307,190y0ol,1.704650e+09
1,NetflixBestOf,[REQUEST] is mindhunter worth it to watch??,285,1276,xk2l0b,1.663761e+09
2,NetflixBestOf,"[DISCUSSION] ""Mindhunter"" deserves a revival o...",133,1707,14kaqdf,1.687864e+09
3,NetflixBestOf,[Discussion] Mindhunter might be best show to ...,151,2506,pqk4kn,1.631962e+09
4,NetflixBestOf,[REQUEST] what is the best Netflix show? BESID...,512,390,130i3o6,1.682593e+09
5,NetflixBestOf,[Request] Looking for good quality crime drama...,468,709,nlgn6l,1.622035e+09
6,NetflixBestOf,[Request] I am really into the vibe that you g...,311,903,mchz8w,1.616623e+09
7,NetflixBestOf,[US] Mindhunter (2019): In the late 1970s two ...,111,2139,cr2sc9,1.565939e+09
8,NetflixBestOf,"[REQUEST] I loved criminal minds, mindhunters,...",241,428,udnz3s,1.651125e+09
9,NetflixBestOf,"Can anyone suggest shows like mr robot, mindhu...",220,368,swbvnz,1.645283e+09


## Step 3.4.3: Search for Comments Mentioning "Mindhunter" (For scenario 1- assume no posts with Minhunter in title were found)
We will search all posts in the two subreddits for comments that mention "Mindhunter."


In [14]:
# Search all comments in a subreddit for mentions of "Mindhunter"
show_title = "Mindhunter"  # The show we're testing
subreddit_name = "NetflixBestOf"  # Choose one subreddit to test
comments_with_show = []  # List to store matching comments

try:
    subreddit = reddit.subreddit(subreddit_name)
    print(f"Searching all comments in subreddit: {subreddit_name}")

    # Stream all comments from the subreddit (can be computationally expensive for large subreddits)
    for comment in subreddit.comments(limit=5000):  # Limit to the most recent 50 comments for testing
        if show_title.lower() in comment.body.lower():  # Check if show is mentioned
            comments_with_show.append({
                'Subreddit': subreddit_name,
                'Comment': comment.body,
                'Upvotes': comment.score,
                'Comment_ID': comment.id,
                'Created_UTC': comment.created_utc
            })
except Exception as e:
    print(f"Error accessing comments in subreddit {subreddit_name}: {e}")

# Convert results to a DataFrame for better readability
if comments_with_show:
    comments_df = pd.DataFrame(comments_with_show)
    print(f"Found {len(comments_df)} comments mentioning '{show_title}':")
    display(comments_df)
else:
    print(f"No comments mentioning '{show_title}' found in subreddit {subreddit_name}.")


Searching all comments in subreddit: NetflixBestOf
Found 3 comments mentioning 'Mindhunter':


,Subreddit,Comment,Upvotes,Comment_ID,Created_UTC
0,NetflixBestOf,Ozark\n\n\nMindhunter \n\n\nStranger Things \n...,2,m81hkvs,1.737318e+09
1,NetflixBestOf,"Just finished Mindhunter and now I’m sad lol, ...",6,m7q80nz,1.737162e+09
2,NetflixBestOf,Just discovered mindhunter. Amazing!,5,m7pwoxk,1.737158e+09
